**Install Required Packages**

In [1]:
import pandas as pd
import rasterio
import geopandas as gpd
import geemap
import ee
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
import fiona
from fiona import listlayers
import geojson
import json
import os

**Landsat-5**

In [3]:
# Function to calculate NDVI
def calculate_ndvi5(image):
    nir = image.select('SR_B4')  # NIR for Landsat 5
    red = image.select('SR_B3')  # Red for Landsat 5
    ndvi = image.expression(
        '(NIR - RED) / (NIR + RED)',
        {
            'NIR': nir,
            'RED': red
        }
    ).rename('NDVI')
    return ndvi
# Function to calculate NDSI
def calculate_ndsi5(image):
    nir = image.select('SR_B4')  # NIR for Landsat 5
    red = image.select('SR_B3')  # Red for Landsat 5
    ndsi = image.expression(
        '(RED - NIR) / (NIR + RED)',
        {
            'NIR': nir,
            'RED': red
        }
    ).rename('NDSI')
    return ndsi
# Function to calculate MNDWI
def calculate_mndwi5(image):
    green = image.select('SR_B2')  # Green for Landsat 5
    swir1 = image.select('SR_B5')  # SWIR1 for Landsat 5
    mndwi = image.expression(
        '(GREEN - SWIR1) / (GREEN + SWIR1)',
        {
            'GREEN': green,
            'SWIR1': swir1
        }
    ).rename('MNDWI')
    return mndwi
# Function to calculate Wetness
def calculate_wet5(image):
    blue = image.select('SR_B1')  # Blue for Landsat 5
    green = image.select('SR_B2')  # Green for Landsat 5
    red = image.select('SR_B3')  # Red for Landsat 5
    nir = image.select('SR_B4')  # NIR for Landsat 5
    swir1 = image.select('SR_B5')  # SWIR1 for Landsat 5
    swir2 = image.select('SR_B7')  # SWIR2 for Landsat 5
    wetness = image.expression(
        '(0.0315*BLUE) +(0.2021*GREEN)+(0.3102*RED)+(0.1594*NIR)-(0.6806*SWIR1)-(0.6109*SWIR2)',
        {
            'BLUE': blue,
            'GREEN': green,
            'RED': red,
            'NIR': nir,
            'SWIR1': swir1,
            'SWIR2': swir2
        }
    ).rename('Wetness')
    return wetness

# Function to calculate NDBSI
def calculate_ndbsi5(image):
    blue = image.select('B1')  # Blue for Landsat 5
    green = image.select('B2')  # Green for Landsat 5
    red = image.select('B3')  # Red for Landsat 5
    nir = image.select('B4')  # NIR for Landsat 5
    swir1 = image.select('B5')  # SWIR1 for Landsat 5
    swir2 = image.select('B7')  # SWIR2 for Landsat 5
    si = image.expression(
        '((SWIR1 + RED) - (NIR + BLUE)) / ((SWIR1 + RED) + (NIR + BLUE))',
        {
            'NIR': nir,
            'RED': red,
            'GREEN': green,
            'SWIR1': swir1,
            'SWIR2': swir2,
            'BLUE': blue
        }
    ).rename('SI')
    ibi = image.expression(
        '(2 * SWIR1 / (SWIR1 + NIR)) - ((NIR / (NIR + RED)) + (GREEN / (GREEN + SWIR1))) / (2 * SWIR1 / (SWIR1 + NIR)) + ((NIR / (NIR + RED)) + (GREEN / (GREEN + SWIR1)))',
        {
            'NIR': nir,
            'RED': red,
            'GREEN': green,
            'SWIR1': swir1,
            'SWIR2': swir2
        }
    ).rename('IBI')
    ndbsi = image.expression(
        '(IBI + SI) / 2',
        {
            'IBI': ibi,
            'SI': si
        }
    ).rename('NDBSI')
    return ndbsi

# Function to calculate LST
def calculate_lst5(image):
    # Step 1: Calculate NDVI
    ndvi = image.normalizedDifference(['B4', 'B3']).rename('NDVI')  # Landsat 5 NIR and Red

    # Step 2: Calculate minimum and maximum NDVI in the AOI
    ndviMin = ee.Number(ndvi.reduceRegion(
        reducer=ee.Reducer.min(),
        geometry=aoi,
        scale=30,
        maxPixels=1e9
    ).values().get(0))

    ndviMax = ee.Number(ndvi.reduceRegion(
        reducer=ee.Reducer.max(),
        geometry=aoi,
        scale=30,
        maxPixels=1e9
    ).values().get(0))

    # Step 3: Calculate Fractional Vegetation (FV)
    fv = ndvi.subtract(ndviMin) \
             .divide(ndviMax.subtract(ndviMin)) \
             .pow(ee.Number(2)) \
             .rename('FV')

    # Step 4: Calculate Emissivity (EM)
    em = fv.multiply(ee.Number(0.004)).add(ee.Number(0.986)).rename('EM')

    # Step 5: Select Thermal Band (Band 6 for Landsat 5)
    thermal = image.select('B6').rename('thermal')

    # Step 6: Calculate Land Surface Temperature (LST)
    lst = thermal.expression(
        '(TB / (1 + (0.00115 * (TB / 1.438)) * log(em))) - 273.15',
        {
            'TB': thermal.select('thermal'),  # Brightness temperature in Kelvin
            'em': em  # Emissivity
        }
    ).rename('LST')
    return lst

In [5]:
# Applies scaling factors.
def apply_scale_factors(image):
  optical_bands = image.select('SR_B.').multiply(0.0000275).add(-0.2)
  thermal_bands = image.select('ST_B6').multiply(0.00341802).add(149.0)
  return image.addBands(optical_bands, None, True).addBands(
      thermal_bands, None, True
  )

**Image Export**

In [7]:
def export_image(image, image_name):
      # Create and start the export task
        task = ee.batch.Export.image.toDrive(
            image=image,
            description=f"{image_name}",
            folder="GEE_Exports",
            fileNamePrefix=image_name,
            region=aoi,
            fileFormat='GeoTIFF',
            scale=30,
            maxPixels=1e13
        )
        task.start()
        print(f"Export task for '{image_name}' started. Check Google Drive for output.")

**Date Format for Google Earth Engine**

In [9]:
from datetime import datetime

def convert_to_ymd(date_str):
    # List of common date formats to try
    date_formats = [
        "%Y-%m-%d",     # 2025-01-04
        "%d-%m-%Y",     # 04-01-2025
        "%m-%d-%Y",     # 01-04-2025
        "%Y/%m/%d",     # 2025/01/04
        "%d/%m/%Y",     # 04/01/2025
        "%m/%d/%Y",     # 01/04/2025
        "%d %b %Y",     # 04 Jan 2025
        "%d %B %Y",     # 04 January 2025
        "%b %d, %Y",    # Jan 04, 2025
        "%B %d, %Y"     # January 04, 2025
    ]
    
    for fmt in date_formats:
        try:
            # Attempt to parse the date string with each format
            parsed_date = datetime.strptime(date_str, fmt)
            # Return the date in the desired format
            return parsed_date.strftime("%Y-%m-%d")
        except ValueError:
            continue  # Try the next format if parsing fails
    
    # If none of the formats match, return an error message
    return "Invalid date format"

In [11]:
print("Type your Google Earth Engine Cloud Project ID")
projectId = input()

Type your Google Earth Engine Cloud Project ID


 ee-jayantakurp17


In [13]:
ee.Authenticate()
ee.Initialize(project=projectId)
Map = geemap.Map(center=(23.715081709283623, 90.08962659013804), zoom=9)
Map

Map(center=[23.715081709283623, 90.08962659013804], controls=(WidgetControl(options=['position', 'transparent_…

In [17]:
print("How you want to call your shapefile?")
print("1. Local Shapefile")
print("2. From Google Earth Engine")
print("3. From Last Drawing in the Map")
cPath = int(input("Your Choice: "))
if cPath == 1:
    filePath = input(r"Enter the path to the shapefile: ").strip('"')
    gdf = gpd.read_file(filePath)
    geojson_str = gdf.to_json()
    geojson_dict = json.loads(geojson_str)
    aoi = ee.FeatureCollection(geojson_dict).geometry()
    Map.addLayer(aoi, {}, 'Area of Interest')
elif cPath == 2:
  filePath = input("Enter Google Earth Engine: ")
  aoi = ee.FeatureCollection(filePath).geometry()
  Map.addLayer(aoi, {}, 'Area of Interest')
elif cPath == 3:
  m = Map.draw_last_feature
  aoi = ee.FeatureCollection(m).geometry()
  Map.addLayer(aoi, {}, 'Area of Interest')
else:
    print("Invalid input")

How you want to call your shapefile?
1. Local Shapefile
2. From Google Earth Engine
3. From Last Drawing in the Map


Your Choice:  1
Enter the path to the shapefile:  "C:\Users\USER\OneDrive - The University of Memphis\Khulna University\Mongla_EcologicalIndex\Shapefile\StudyArea.shp"


In [37]:
startDate = input("Enter Start date: ")
endDate = input("Enter End date: ")
sDate = convert_to_ymd(startDate)
eDate = convert_to_ymd(endDate)

Enter Start date:  1-1-2010
Enter End date:  30-12-2010


In [21]:
cCover = float(input("Could Coverage: "))

Could Coverage:  5


In [39]:
sImage = ee.ImageCollection('LANDSAT/LT05/C02/T1_L2') \
                        .filterBounds(aoi) \
                        .filterDate(sDate, eDate) \
                        .filterMetadata('CLOUD_COVER', 'less_than', cCover)
sImage5 = sImage.median().clip(aoi)
sImage5 = apply_scale_factors(sImage5)
Map.addLayer(sImage5, {'bands': ['SR_B3', 'SR_B3', 'SR_B1'], 'min': 0, 'max': 0.3}, 'Landsat-5 Image')

In [40]:
thImage = ee.ImageCollection('LANDSAT/LT05/C02/T1_TOA') \
                        .filterBounds(aoi) \
                        .filterDate(sDate, eDate) \
                        .filterMetadata('CLOUD_COVER', 'less_than', cCover)
thImage5 = thImage.median().clip(aoi)
Map.addLayer(thImage5, {'bands': ['B3', 'B3', 'B1'], 'min': 0, 'max': 0.3}, 'Landsat-5 thImage')

In [41]:
ndvi = calculate_ndvi5(sImage5)
wetness = calculate_wet5(sImage5)
ndbsi = calculate_ndbsi5(thImage5)
lst = calculate_lst5(thImage5)
mndwi = calculate_mndwi5(sImage5)
export_image(ndvi, "ndvi_00")
export_image(wetness, "wetness_00")
export_image(ndbsi, "ndbsi_00")
export_image(lst, "lst_00")
export_image(mndwi, "mndwi_10")

Export task for 'ndvi_00' started. Check Google Drive for output.
Export task for 'wetness_00' started. Check Google Drive for output.
Export task for 'ndbsi_00' started. Check Google Drive for output.
Export task for 'lst_00' started. Check Google Drive for output.


In [43]:
ndsi = calculate_ndsi5(sImage5)
export_image(ndsi, "ndsi_10")

Export task for 'ndsi_10' started. Check Google Drive for output.
